## 서브그래프

랭그래프에서는 하나의 노드가 그래프인 하위그래프를 사용할 수 있음.<br>
단, 마구만들 경우 복잡한 워크플로우가 되므로 주의<br>
독립적으로 잘 작동하는 하나의 워크플로우로 구성 및 상위 워크플로우에서 재사용<br>

해당 내용은 요즘 ai 에이전트 개발 llM RAG ADK MCP LangChain A2A (저자: 박승규) 에서 첨부하였습니다.

In [3]:
# 필요한 라이브러리 설치
import httpx
from geopy.geocoders import Nominatim
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain.chat_models import init_chat_model
from typing import Literal
import json

In [11]:
def get_coordinates(city_name: str) -> tuple[float, float]:
    """도시 이름을 받아 위도와 경도를 반환합니다."""
    geolocator = Nominatim(user_agent="weather_app_langgraph", timeout=10)
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    raise ValueError(f"좌표를 찾을 수 없습니다: {city_name}")

In [5]:
def get_weather(city_name: str) -> str:
    """도시 이름을 받아 해당 도시의 현재 날씨 정보를 반환합니다."""
    print(f"날씨 조회: {city_name}")
    latitude, longitude = get_coordinates(city_name)
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    response = httpx.get(url)
    response.raise_for_status()
    return json.dumps(response.json())

In [6]:
# ① 기상 전문가 서브그래프 생성
def create_weather_agent():
    """날씨 관련 질문을 처리하는 전문가 서브그래프를 생성합니다."""
    model = init_chat_model("gpt-5-mini", model_provider="openai").bind_tools(
        [get_weather]
    )
    tool_node = ToolNode([get_weather])

    def call_model(state: MessagesState):
        return {"messages": [model.invoke(state["messages"])]}

    graph = StateGraph(MessagesState)
    graph.add_node("call_model", call_model)
    graph.add_node("tool_node", tool_node)

    graph.add_edge(START, "call_model")
    graph.add_conditional_edges(
        "call_model",
        lambda s: "tool_node" if s["messages"][-1].tool_calls else END,
        {"tool_node": "tool_node", END: END},
    )
    graph.add_edge("tool_node", "call_model")
    return graph.compile()


In [8]:

def router(state: MessagesState) -> Literal["weather_expert", "general_agent"]:
    query = state["messages"][-1].content.lower()
    if "날씨" in query or "기온" in query:
        print("라우팅 결정: 기상 전문가에게 위임")
        return "weather_expert"
    print("라우팅 결정: 일반 에이전트가 처리")
    return "general_agent"

In [9]:
# ② 메인 그래프 생성
def create_main_agent(weather_subgraph):
    """질문을 라우팅하고 처리하는 메인 에이전트 그래프를 생성합니다."""
    main_model = init_chat_model("gpt-5-mini", model_provider="openai")

    workflow = StateGraph(MessagesState)
    workflow.add_node(
        "general_agent", lambda s: {"messages": [main_model.invoke(s["messages"])]}
    )
    workflow.add_node("weather_expert", weather_subgraph)
    workflow.add_conditional_edges(
        START,
        router,
        {
            "weather_expert": "weather_expert",
            "general_agent": "general_agent",
        },
    )
    workflow.add_edge("general_agent", END)
    workflow.add_edge("weather_expert", END)
    return workflow.compile()

In [12]:
if __name__== "__main__":
    print("=== LangGraph 서브그래프 예제 (기상 전문가) ===\n")
    weather_agent = create_weather_agent()
    main_agent = create_main_agent(weather_agent)
    
    queries = ["성남 날씨 어때?", "잠은 몇시간 자는게 좋을까?"]
    for query in queries:
        print(f"\n--- 질문: {query} ---")
        result = main_agent.invoke({"messages": [HumanMessage(content=query)]})
        print(f"최종 답변: {result['messages'][-1].content}")
        print("-" * 20)

=== LangGraph 서브그래프 예제 (기상 전문가) ===


--- 질문: 성남 날씨 어때? ---
라우팅 결정: 기상 전문가에게 위임
날씨 조회: 성남
최종 답변: 지금 성남은 맑고 추운 날씨입니다.

- 시간(측정): 2026-01-27 16:30 (KST, 원자료 2026-01-27T07:30 UTC)
- 기온: -2.9°C
- 하늘상태: 맑음 (weather code 0)
- 바람: 14.5 km/h, 북서(약 305°)
- 강수: 없음

한마디 조언: 바람이 있어 체감온도가 더 낮게 느껴질 수 있으니 외출하실 때는 두꺼운 코트와 목도리·장갑 챙기세요. 더 자세한 예보(오늘 낮/밤 기온, 강수 확률 등) 원하시면 말씀해 주세요.
--------------------

--- 질문: 잠은 몇시간 자는게 좋을까? ---
라우팅 결정: 일반 에이전트가 처리
최종 답변: 간단히 말하면 나이마다 권장 수면 시간이 다르고 개인 차가 큽니다. 일반 권장치는 다음과 같습니다(미국 수면재단 등 권고 기준):

- 신생아(0–3개월): 14–17시간  
- 영아(4–11개월): 12–15시간  
- 유아(1–2세): 11–14시간  
- 미취학(3–5세): 10–13시간  
- 학령기(6–13세): 9–11시간  
- 청소년(14–17세): 8–10시간  
- 성인(18–64세): 7–9시간  
- 노인(65세 이상): 7–8시간

주의할 점
- 개인차: 일부는 6시간으로도 괜찮고, 일부는 10시간 가까이 필요합니다. 중요한 건 수면 시간뿐 아니라 낮 동안의 피로감, 집중력, 기분 등 기능이 정상인지입니다.
- 과소/과다 수면 둘 다 건강에 영향(심혈관·대사·정신건강 등)을 줄 수 있습니다.

수면의 질을 높이는 간단한 팁
- 매일 같은 시간에 자고 일어나기(주말에도 규칙성 유지)  
- 잠자리 환경: 어둡고 조용하며 서늘하게(약 16–19°C 권장)  
- 취침 전 1시간 정도 화면(스마트폰·TV) 사용 줄이기  
- 카페인·니코틴 섭취는 오후 늦게 피하기, 과